# VacationPy
----




------------------------------------
- This code took about 18 seconds to run
- i've ran it a couple times and there seems to be a bias around africa, the middle east, and india. It could be caused by the random coordinates generated in weather.py or the fact that these regions are huge, or it could be the weather based filters I applied

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

from full_country import full_name_dict

### this allows me to keep my keys in a folder outside my repo
import sys
try:
    sys.path.insert(1,'/Users/Trav/Desktop/Coding/KEY_PACKAGE')
    from api_keys import g_key
except:
    from api_keys import g_key





### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
## import csv to dataframe, add try block to deal with directory issues
csv_import_path = os.path.join('..','Weather','Export_Folder','city_data.csv')
city_weather_df=pd.read_csv(csv_import_path)

city_weather_df.head()

# # ### using this to limit the request, remove once completed
# city_weather_df=city_weather_df.head()
# city_weather_df.head(20)

,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp
0,-68.26,163.04,bluff,nz,"(-46.6, 168.333333)",-46.600000,168.333333,48.00,80,100,8.01,48.00
1,-48.48,-120.05,rikitea,pf,"(-23.1202778, -134.9691667)",-23.120278,-134.969167,79.93,69,93,8.50,79.93
2,41.17,50.96,artyom,az,"(40.472255, 50.333041)",40.472255,50.333041,42.80,93,75,11.50,42.80
3,59.48,65.43,lugovoy,ru,"(59.733333, 65.833333)",59.733333,65.833333,17.58,97,80,5.68,17.58
4,-39.10,67.59,souillac,mu,"(-20.5166667, 57.5166667)",-20.516667,57.516667,77.00,88,75,5.75,77.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
#configure gmaps with key
gmaps.configure(api_key=g_key)

In [9]:

#defines coordinates to plot and weight is humididty

locations=city_weather_df[['Latitude','Longitude']]
weighty=city_weather_df['Humidity']

fig=gmaps.figure(center=(0,0),zoom_level=2)
heat=gmaps.heatmap_layer(locations,weights=weighty,dissipating=True,max_intensity=200,point_radius=20)
fig.add_layer(heat)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
#narrows down the dataframe to a dataframe with ideal weather
ideal_weather = city_weather_df.loc[(city_weather_df['Max Temp']>70)&(city_weather_df['Max Temp']<80)&(city_weather_df['Wind Speed']<10)&(city_weather_df['Cloudiness']==0),:]

#drops bad data
ideal_weather.dropna()


ideal_weather.head()

,Approximate Lat,Approximate Lon,City Name,Country Code,Exact Coordinates,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp
12,-21.78,-25.20,caravelas,br,"(-17.75, -39.25)",-17.750000,-39.250000,76.86,82,0,6.20,76.86
31,-48.21,69.61,saint-philippe,re,"(-21.35, 55.7666667)",-21.350000,55.766667,74.79,69,0,6.91,78.80
106,12.91,-27.27,sao filipe,cv,"(14.9, -24.5166667)",14.900000,-24.516667,74.86,65,0,5.70,74.86
140,9.58,-20.57,oussouye,sn,"(12.485, -16.5469444)",12.485000,-16.546944,71.56,78,0,2.30,73.40
209,8.05,4.49,ilobu,ng,"(7.833333, 4.483333)",7.833333,4.483333,74.37,85,0,4.68,74.37


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
#simplifies and defines the hotel_df
hotel_df=ideal_weather.drop(['Approximate Lat','Approximate Lon','Exact Coordinates'],axis=1).reset_index(drop=True)

#creates blank series that will be filled in
hotel_df['Hotel Name']=""
hotel_df['Hotel Lat']=""
hotel_df['Hotel Lon']=""
# #renaming and making uppercase
# hotel_df=hotel_df.rename(columns={'City Name':'City','Country Code':'Country'})
hotel_df['Country Code']=hotel_df['Country Code'].str.upper()

hotel_df=hotel_df.rename(columns={'City Name':'City'})
#creating a dataframe from the countries full names
country_name_df=pd.DataFrame.from_dict(full_name_dict,orient='index').reset_index()
country_name_df=country_name_df.rename(columns={'index':'Country Code',0:'Country'})

#merging in the country names

hotel_df=pd.merge(country_name_df,hotel_df,on="Country Code",how='right')

hotel_df.head()

,Country Code,Country,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Max Temp,Hotel Name,Hotel Lat,Hotel Lon
0,AU,Australia,port hedland,-20.312149,118.610588,78.80,94,0,6.91,78.80,,,
1,BR,Brazil,caravelas,-17.750000,-39.250000,76.86,82,0,6.20,76.86,,,
2,CV,Cape Verde,sao filipe,14.900000,-24.516667,74.86,65,0,5.70,74.86,,,
3,NG,Nigeria,ilobu,7.833333,4.483333,74.37,85,0,4.68,74.37,,,
4,RE,Réunion,saint-philippe,-21.350000,55.766667,74.79,69,0,6.91,78.80,,,


In [13]:

### finds the closest hotel and fills in the hotel name and coordinates to the dataframe
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "Hotel"
target_radius = 5000
target_type = "Hotel"

for i,r in hotel_df.iterrows():
    c_lat=hotel_df['Latitude'][i]
    c_lon = hotel_df['Longitude'][i]
    target_coordinates =f'{c_lat}, {c_lon}' 
    
    param = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    places_json=requests.get(base_url,params=param).json()
    ## the city tessalit is literally in the middle of the sahara desert and returns no results, so this is a work around.

    if places_json['status']=='ZERO_RESULTS':
        hotel_df['Hotel Name'][i]=np.nan
    else:
        hotel_df['Hotel Name'][i]=places_json["results"][0]['name']
        hotel_df['Hotel Lat'][i]=places_json["results"][0]['geometry']['location']['lat']
        hotel_df['Hotel Lon'][i]=places_json["results"][0]['geometry']['location']['lng']


## Dropping the cities that returned no results 
hotel_df=hotel_df.dropna()

hotel_df.head(30)


IndexError: list index out of range

In [ ]:
# # #create a new folder and exports the dataframe

def dataframe_to_csv(df):
    current_directory=os.getcwd()
    new_folder_path =os.path.join(current_directory,r'Export_Folder')
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
    csv_export_path = os.path.join(new_folder_path,'hotel_data.csv')
    df.to_csv(csv_export_path,index=False)
dataframe_to_csv(hotel_df)

In [ ]:
## import the csv file you just saved
def import_path():
    current_directory=os.getcwd()
    import_path.new_folder_path =os.path.join(current_directory,r'Export_Folder')
    csv_export_path = os.path.join(import_path.new_folder_path,'hotel_data.csv')
    return csv_export_path

im_path=import_path()
csv_imp_df=pd.read_csv(im_path,delimiter=',')
csv_imp_df.head()

In [ ]:


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in csv_imp_df.iterrows()]

# hotel_info


In [ ]:
### plotting heat map and hotel locations
y_locations=city_weather_df[['Latitude','Longitude']]
x_locations=csv_imp_df[['Hotel Lat','Hotel Lon']]
x_weighty=city_weather_df['Humidity']



x_fig=gmaps.figure(center=(0,0),zoom_level=2)
x_heat=gmaps.heatmap_layer(y_locations,weights=x_weighty,dissipating=True,max_intensity=200,point_radius=20)
x_marks=gmaps.marker_layer(x_locations,info_box_content=hotel_info)
x_fig.add_layer(x_marks)
x_fig.add_layer(x_heat)

x_fig